<a href="https://colab.research.google.com/github/irynagorbenko/data-science/blob/opi-radiomics/logistic_regression_all_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Read data from Google Drive
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()
import mlxtend
gc = gspread.authorize(creds)
from google.colab import files

# Open our new sheet and add some data.
spreadsheet_t2w = gc.open_by_url('https://docs.google.com/spreadsheets/d/1PTjiajpfNcK3_S-t1_tZNT4Sr6dYjMaFzpQgS9FvYUU/edit#gid=0')
rad_t2w= spreadsheet_t2w.worksheet('final_rad');

spreadsheet_sr = gc.open_by_url('https://docs.google.com/spreadsheets/d/1VX48Vb9cYDHRT1iOfIrpKW2duWPGR6zguk5rGX5UeX0/edit#gid=0')
sr= spreadsheet_sr.worksheet('final_SR');

# get_all_values gives a list of rows.
# rows = rad_t2w.get('A1:AU1028')

In [2]:
import pandas as pd
radDF = pd.DataFrame(rad_t2w.get_all_records())
radDF.shape

(976, 47)

In [3]:
srDF = pd.DataFrame(sr.get_all_records())
srDF.shape

(976, 68)

In [ ]:
radDF.head()

In [ ]:
radDF.info()

In [ ]:
srDF.info()

In [ ]:
x = radDF[['original_firstorder_Energy']]
y = srDF[['lesion_pirads']]
idx = y[y['lesion_pirads'] == ''].index # empty pi-rads in line 183, 221, 796
idx

Int64Index([183, 221, 796], dtype='int64')

In [ ]:
x = x.drop(idx) # independent variable - input
y = y.drop(idx) # dependent variable - output

In [ ]:
x.shape

(973, 1)

# **Distribution check**




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# matplotlib histogram
# plt.hist(x, color = 'blue', edgecolor = 'black', bins = 50)#, bins = int(180/5))

# seaborn histogram
sns.distplot(x, hist=True, kde=False, bins = 80, color = 'blue', hist_kws={'edgecolor':'black'}) #bins=int(180/5), 

# Add labels
plt.title('Histogram of original_firstorder_Energy')
plt.xlabel('Energy')
plt.ylabel('Count')

In [ ]:
srDF[['patient_id', 'radiologist_id', 'patient_pirads']]

In [ ]:
import seaborn as sns

temprad = radDF[['patient_id', 'radiologist_id', 'original_firstorder_Energy']]
tempsr = srDF[['patient_id', 'radiologist_id', 'patient_pirads']]
idx = tempsr[tempsr['patient_pirads'] == ''].index
temprad = temprad.drop(idx)
tempsr = tempsr.drop(idx)

tempDF = pd.merge(temprad, tempsr, on=["patient_id", "radiologist_id"])

sns.scatterplot(data=tempDF, x="original_firstorder_Energy", y="patient_pirads")#, hue="time")

In [ ]:
# Check how many studies of pi-rads in range 1 to 5 are in data

pirads1 = tempDF['patient_pirads'].value_counts()[1]
pirads2 = tempDF['patient_pirads'].value_counts()[2]
pirads3 = tempDF['patient_pirads'].value_counts()[3]
pirads4 = tempDF['patient_pirads'].value_counts()[4]
pirads5 = tempDF['patient_pirads'].value_counts()[5]
print(pirads1, pirads2, pirads3, pirads4, pirads5)

3 32 118 468 631


In [ ]:
sns.boxplot(data=tempDF, x="patient_pirads", y="original_firstorder_Energy")

In [ ]:
x = x.to_numpy()
y = y.to_numpy().ravel()
y = y.astype(str).astype(float).astype(int)

In [ ]:
# Authors: Tom Dupre la Tour <tom.dupre-la-tour@m4x.org>
# License: BSD 3 clause
#!pip install -U scikit-learn
#!pip install --upgrade scikit-learn

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import DecisionBoundaryDisplay

# make 3-class dataset for classification
#centers = [[-5, 0], [0, 1.5], [5, -1]]
#X, y = make_blobs(n_samples=1000, centers=centers, random_state=40)
#transformation = [[0.4, 0.2], [-0.4, 1.2]]
#X = np.dot(X, transformation)

for multi_class in ("multinomial", "ovr"):
    clf = LogisticRegression(solver="sag", max_iter=100, random_state=42, multi_class=multi_class).fit(x, y)

    # print the training scores
    print("training score : %.3f (%s)" % (clf.score(x, y), multi_class))

    _, ax = plt.subplots()
    DecisionBoundaryDisplay.from_estimator(clf, x, response_method="predict", cmap=plt.cm.Paired, ax=ax)
    plt.title("Decision surface of LogisticRegression (%s)" % multi_class)
    plt.axis("tight")

    # Plot also the training points
    colors = "bry"
    for i, color in zip(clf.classes_, colors):
        idx = np.where(y == i)
        plt.scatter(
            x[idx, 0], x[idx, 1], c=color, cmap=plt.cm.Paired, edgecolor="black", s=20
        )

    # Plot the three one-against-all classifiers
    xmin, xmax = plt.xlim()
    ymin, ymax = plt.ylim()
    coef = clf.coef_
    intercept = clf.intercept_

    def plot_hyperplane(c, color):
        def line(x0):
            return (-(x0 * coef[c, 0]) - intercept[c]) / coef[c, 1]

        plt.plot([xmin, xmax], [line(xmin), line(xmax)], ls="--", color=color)

    for i, color in zip(clf.classes_, colors):
        plot_hyperplane(i, color)

plt.show()


#**scikit-learn**










In [ ]:
# Import libraries for analysis in scikit-learn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# 1. Example data

# The input and output should be NumPy arrays
#x = np.arange(10).reshape(-1, 1) # The array x is required to be two-dimensional. It should have one column for each input, and the number of rows should be equal to the number of observations. 
#y = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1]) # y is one-dimensional with ten items. Each item corresponds to one observation. It contains only zeros and ones since this is a binary classification problem.

x = x.to_numpy()
#y = y.to_numpy()

y = y.to_numpy().ravel()

In [ ]:
y = y.astype(str).astype(float).astype(int)

In [ ]:
# 2. Model

model = LogisticRegression(solver='saga')
model.fit(x, y) # it fits the model and returns the model instance
# Important parameters

# penalty is a string ('l2' by default) that decides whether there is regularization and which approach to use. Other options are 'l1', 'elasticnet', and 'none'.
# C is a positive floating-point number (1.0 by default) that defines the relative strength of regularization. Smaller values indicate stronger regularization.
# class_weight is a dictionary, 'balanced', or None (default) that defines the weights related to each class. When None, all classes have the weight one.
# solver is a string ('liblinear' by default) that decides what solver to use for fitting the model. Other options are 'newton-cg', 'lbfgs', 'sag', and 'saga'.
# max_iter is an integer (100 by default) that defines the maximum number of iterations by the solver during model fitting.
# multi_class is a string ('ovr' by default) that decides the approach to use for handling multiple classes. Other options are 'multinomial' and 'auto'.
# warm_start is a Boolean (False by default) that decides whether to reuse the previously obtained solution.
# n_jobs is an integer or None (default) that defines the number of parallel processes to use. None usually means to use one core, while -1 means to use all available cores.
# l1_ratio is either a floating-point number between zero and one or None (default). It defines the relative importance of the L1 part in the elastic-net regularization.

# Warning
# 'liblinear' solver doesn’t work without regularization.
#'newton-cg', 'sag', 'saga', and 'lbfgs' don’t support L1 regularization.
#'saga' is the only solver that supports elastic-net regularization.

In [ ]:
# the example of binary classification, and y can be 0 or 1
model.classes_ 

In [ ]:
# the intercept 𝑏₀ of the linear function 𝑓
model.intercept_

In [ ]:
# the value of the slope 𝑏₁ of the linear function 𝑓
model.coef_

In [ ]:
# 3. Evaluate the Model

model.predict_proba(x) # returns the matrix of probabilities that the predicted output is equal to zero or one

# Each row corresponds to a single observation. 
# The first column is the probability of the predicted output being zero, that is 1 - 𝑝(𝑥). 
# The second column is the probability that the output is one, or 𝑝(𝑥).

In [ ]:
# The actual predictions, based on the probability matrix and the values of 𝑝(𝑥)
model.predict(x)

In [ ]:
# Returns the ratio of the number of correct predictions to the number of observations
model.score(x, y)

0.45426515930113054

In [ ]:
# Confusion matrix

#In the case of binary classification, the confusion matrix shows the numbers of the following:
# True negatives in the upper-left position
# False negatives in the lower-left position
# False positives in the upper-right position
# True positives in the lower-right position

confusion_matrix(y, model.predict(x))
# Three true negative predictions: The first three observations are zeros predicted correctly.
# No false negative predictions: These are the ones wrongly predicted as zeros.
# One false positive prediction: The fourth observation is a zero that was wrongly predicted as one.
# Six true positive predictions: The last six observations are ones predicted correctly.

In [ ]:
# Visualization of the confusion matrix

cm = confusion_matrix(y, model.predict(x))

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cm)
ax.grid(False)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='red')
plt.show()

In [ ]:
# Classification report

print(classification_report(y, model.predict(x)))

#  It returns a report on the classification as a dictionary if you provide output_dict=True or a string otherwise.

In [ ]:
# 4. Improve the Model

# regularization strength C equal to 10.0, instead of the default value of 1.0
# the larger value of C means weaker regularization, or weaker penalization related to high values of 𝑏₀ and 𝑏₁.
model = LogisticRegression(solver='saga')#, class_weight='balanced') # C=10.0,
model.fit(x, y)
model.score(x, y)

In [ ]:
confusion_matrix(y, model.predict(x))

In [ ]:
print(classification_report(y, model.predict(x)))

# **StatsModels**



In [ ]:
# Import libraries
import numpy as np
import statsmodels.api as sm

In [ ]:
# 1. DATA

# StatsModels doesn’t take the intercept 𝑏₀ into account, and you need to include the additional column of ones in x.
x = np.arange(10).reshape(-1, 1)
y = np.array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1])
x = sm.add_constant(x) # add_constant() takes the array x as the argument and returns a new array with the additional column of ones. 
# The first column of x corresponds to the intercept 𝑏₀. The second column contains the original values of x.

In [ ]:
# 2. MODEL

# Create a Model and Train It
model = sm.Logit(y, x) # Note that the first argument here is y, followed by x.
result = model.fit(method='newton')

In [ ]:
# obtain the values of 𝑏₀ and 𝑏₁. The first element is the intercept 𝑏₀, the second is the slope 𝑏₁.
result.params

In [ ]:
# 3. Evaluate the Model

result.predict(x)

In [ ]:
# Actual predicted outputs
(result.predict(x) >= 0.5).astype(int)

In [ ]:
# Confusion matrix
result.pred_table()

In [ ]:
# Summary

result.summary()

In [ ]:
result.summary2()

# **Gradient descent algorithm**

In [ ]:
x = x.to_numpy()
y = y.to_numpy().ravel()
y = y.astype(str).astype(float).astype(int)

In [ ]:
X = x
Y = y

In [ ]:
from sklearn.datasets import load_iris
X = load_iris().data # numpy.ndarray
Y = load_iris().target # numpy.ndarray

In [ ]:
# https://towardsdatascience.com/multiclass-logistic-regression-from-scratch-9cc0007da372

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax
onehot_encoder = OneHotEncoder(sparse=False)
from sklearn.datasets import load_iris

def loss(X, Y, W):
    """
    Y: onehot encoded
    """
    Z = - X @ W
    N = X.shape[0]
    loss = 1/N * (np.trace(X @ W @ Y.T) + np.sum(np.log(np.sum(np.exp(Z), axis=1))))
    return loss

def gradient(X, Y, W, mu):
    """
    Y: onehot encoded 
    """
    Z = - X @ W
    P = softmax(Z, axis=1)
    N = X.shape[0]
    gd = 1/N * (X.T @ (Y - P)) + 2 * mu * W
    return gd

def gradient_descent(X, Y, max_iter=1000, eta=0.1, mu=0.01):
    """
    Very basic gradient descent algorithm with fixed eta and mu
    """
    Y_onehot = onehot_encoder.fit_transform(Y.reshape(-1,1))
    W = np.zeros((X.shape[1], Y_onehot.shape[1]))
    step = 0
    step_lst = [] 
    loss_lst = []
    W_lst = []
 
    while step < max_iter:
        step += 1
        W -= eta * gradient(X, Y_onehot, W, mu)
        step_lst.append(step)
        W_lst.append(W)
        loss_lst.append(loss(X, Y_onehot, W))

    df = pd.DataFrame({
        'step': step_lst, 
        'loss': loss_lst
    })
    return df, W

class Multiclass:
    def fit(self, X, Y):
        self.loss_steps, self.W = gradient_descent(X, Y)

    def loss_plot(self):
        return self.loss_steps.plot(
            x='step', 
            y='loss',
            xlabel='step',
            ylabel='loss'
        )

    def predict(self, H):
        Z = - H @ self.W
        P = softmax(Z, axis=1)
        return np.argmax(P, axis=1)
    
#X = load_iris().data
#Y = load_iris().target

# fit model
model = Multiclass()
model.fit(X, Y)

# plot loss
model.loss_plot()

# predict 
model.predict(X)

# check the predicted value and the actual value
model.predict(X) == Y

# **Logistics regression**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='saga', max_iter=2000, multi_class='multinomial',random_state=0).fit(x, y)
logreg.score(x,y)

In [ ]:
print('intercept ', logreg.intercept_[0])
print('classes', logreg.classes_)

# **Multiclass Receiver Operating Characteristic (ROC)**

In [ ]:
X = radDF[[1,1]]

In [ ]:
# Load and prepare data
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Radiomic First Order Features
X = radDF[5:22, :]

iris = load_iris()
target_names = iris.target_names
X, y = iris.data, iris.target
y = iris.target_names[y]

random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
n_classes = len(np.unique(y))
X = np.concatenate([X, random_state.randn(n_samples, 200 * n_features)], axis=1)
(
    X_train,
    X_test,
    y_train,
    y_test,
) = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

In [ ]:
n_classes

In [ ]:
# Train a LogisticRegression model
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
y_score = classifier.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
# One-vs-Rest multiclass ROC

from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
label_binarizer.transform(["virginica"])

In [ ]:
class_of_interest = "virginica"
class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
class_id

In [ ]:
# Show ROC curve
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()